Import libraries 

In [5]:
import folium
import networkx as nx
import os
import webbrowser
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
import requests

In [2]:
data_series = pd.read_csv("../data/interim/bicimad_time_series.csv", sep=',')

In [3]:
data_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30001 entries, 0 to 30000
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   idDriver                  15725 non-null  object 
 1   idBike                    30001 non-null  int64  
 2   trip_minutes              30001 non-null  float64
 3   fleet                     30001 non-null  int64  
 4   unlock_date               30001 non-null  object 
 5   unlock_hour               30001 non-null  int64  
 6   latitude_unlock           30001 non-null  float64
 7   longitude_unlock          30001 non-null  float64
 8   station_unlock            30001 non-null  int64  
 9   dock_unlock               30001 non-null  int64  
 10  unlock_station_name       30001 non-null  object 
 11  lock_date                 30001 non-null  object 
 12  lock_hour                 30001 non-null  int64  
 13  latitude_lock             30001 non-null  float64
 14  longit

# Bloque 1: Crear el Grafo

In [4]:
# Bloque 1: Crear el Grafo
G = nx.DiGraph()

# Agregar nodos al grafo (estaciones)
for index, row in data_series.iterrows():
    G.add_node(row['station_unlock'], latitude=row['latitude_unlock'], longitude=row['longitude_unlock'])
    G.add_node(row['station_lock'], latitude=row['latitude_lock'], longitude=row['longitude_lock'])

# Agregar arcos ponderados al grafo (distancias entre estaciones)
for index, row in data_series.iterrows():
    G.add_edge(row['station_unlock'], row['station_lock'], distance=row['distance'], time=row['trip_minutes'])

# Bloque 2: Funciones Auxiliares

Bicicletas disponibles según estación

In [6]:

# URL base de la API
base_url = "https://openapi.emtmadrid.es/v1"

def iniciar_sesion(email, password):
    url = f"{base_url}/mobilitylabs/user/login/"
    headers = {
        "email": email,
        "password": password
    }

    response = requests.get(url, headers=headers)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Capturar y devolver el token de acceso
        token = response.json().get("data", [{}])[0].get("accessToken")
        return token
    else:
        # Imprimir el código de estado y la respuesta en caso de error
        print("Error en la solicitud de inicio de sesión:")
        print("Código de estado:", response.status_code)
        print("Respuesta:", response.json())
        return None

def obtener_estado_estacion_bicimad(access_token, id_station):
    url = f"{base_url}/transport/bicimad/stations/{id_station}/"
    headers = {"accessToken": access_token}

    response = requests.get(url, headers=headers)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Capturar y devolver los detalles de la estación
        estado_estacion = response.json()
        return estado_estacion
    else:
        # Imprimir el código de estado y la respuesta en caso de error
        print("Error en la solicitud de estado de estación BiciMAD:")
        print("Código de estado:", response.status_code)
        print("Respuesta:", response.json())
        return None

def obtener_estado_estacion_bicimad_por_id_o_nombre(access_token, id_o_nombre):
    url = f"{base_url}/transport/bicimad/stations/"
    headers = {"accessToken": access_token}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lanza una excepción para códigos de estado de error

        # Verificar si la solicitud fue exitosa (código de estado 200)
        if response.status_code == 200:
            # Capturar y devolver la lista de estaciones
            lista_estaciones = response.json().get("data", [])

            # Si se proporciona un ID válido, buscar la estación por ID
            if id_o_nombre.isdigit():
                estacion_por_id = [estacion for estacion in lista_estaciones if estacion.get('id') == int(id_o_nombre)]
                return estacion_por_id
            # Si se proporciona un nombre, buscar estaciones que coincidan parcialmente con el nombre
            else:
                estaciones_coincidentes = [estacion for estacion in lista_estaciones if id_o_nombre.lower() in estacion.get('name', '').lower()]
                return estaciones_coincidentes
        else:
            # Imprimir el código de estado y la respuesta en caso de error
            print("Error en la solicitud de lista de estaciones BiciMAD:")
            print("Código de estado:", response.status_code)
            print("Respuesta:", response.json())
            return None
    except requests.exceptions.RequestException as e:
        print("Error en la solicitud de lista de estaciones BiciMAD:", str(e))
        return None

def mostrar_estacion_en_mapa(estacion):
    if estacion:
        coordenadas = estacion.get('geometry', {}).get('coordinates', [0, 0])
        nombre_estacion = estacion.get('name', 'N/A')

        # Cambio en la línea siguiente
        mapa = folium.Map(location=[coordenadas[1], coordenadas[0]], zoom_start=15, tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', attr='Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')
        folium.Marker(location=[coordenadas[1], coordenadas[0]], popup=nombre_estacion).add_to(mapa)
        
        # Guardar el mapa como HTML
        mapa.save("mapa_estacion.html")
        
        # Abrir el mapa en el navegador predeterminado
        webbrowser.open_new_tab("mapa_estacion.html")

        print(f"Se ha generado un mapa para la estación '{nombre_estacion}'.")
    else:
        print("No se pudo obtener la información de la estación.")


# Test Inicio exitoso
if __name__ == "__main__":
    # Credenciales de inicio de sesión
    email = "ruben.c_ac@icloud.com"
    password = "Prada2024!"

    # Iniciar sesión y obtener el token
    token = iniciar_sesion(email, password)

    # Continuar solo si se obtuvo el token con éxito
    if token:
        print("Inicio de sesión exitoso. Token de acceso:", token)

        # Solicitar al usuario que ingrese el ID o nombre de la estación
        id_o_nombre_estacion = input("Ingrese el ID o parte del nombre de la estación BiciMAD: ")

        # Consultar la estación BiciMAD por ID o nombre
        estacion_resultado = obtener_estado_estacion_bicimad_por_id_o_nombre(token, id_o_nombre_estacion)

        # Imprimir la respuesta
        if estacion_resultado:
            print(f"Detalles de la estación BiciMAD:")
            for estacion in estacion_resultado:
                print(f"ID: {estacion['id']}, Nombre: {estacion['name']}, Bicicletas disponibles: {estacion['dock_bikes']}")
                mostrar_estacion_en_mapa(estacion)
        else:
            print("No se encontraron estaciones que coincidan con el ID o nombre proporcionado.")

Inicio de sesión exitoso. Token de acceso: 19ca74d2-2f42-4e61-a605-90eecc0861f9
Detalles de la estación BiciMAD:
ID: 2097, Nombre: 360 - Avda. Nuestra Señora de Fatima, 34, Bicicletas disponibles: 4
Se ha generado un mapa para la estación '360 - Avda. Nuestra Señora de Fatima, 34'.


Hora punta

In [5]:
def get_time_adjustment(current_time, current_day_of_week):
    # Ejemplo: Ajustar el tiempo de viaje en horas pico (de 7 a 9 a.m. y de 5 a 7 p.m. de lunes a viernes)
    weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
    peak_hours = (7 <= current_time <= 9) or (17 <= current_time <= 19)

    if current_day_of_week in weekdays and peak_hours:
        return 1.5  # Ajuste del 50% para horas pico

    return 1.0  # Sin ajuste por defecto

Estación más cercana a mi ubicación

In [6]:
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd

# Definimos los límites de Madrid
madrid_bounds = {'west': -3.889004, 'south': 40.312071, 'east': -3.518011, 'north': 40.643523}

def create_station_data(data_series):
    if madrid_bounds['west'] <= data_series['longitude_unlock'] <= madrid_bounds['east'] and madrid_bounds['south'] <= data_series['latitude_unlock'] <= madrid_bounds['north']:
        return {
            'station_id': data_series['unlock_station_name_n'],
            'latitude': data_series['latitude_unlock'],
            'longitude': data_series['longitude_unlock'],
            'address': data_series['station_unlock']
        }
    else:
        return None

def get_user_location():
    while True:
        address = input("Ingrese su dirección: ")
        geolocator = Nominatim(user_agent="geo_locator")
        location = geolocator.geocode(address)

        if location and madrid_bounds['west'] <= location.longitude <= madrid_bounds['east'] and madrid_bounds['south'] <= location.latitude <= madrid_bounds['north']:
            return location.latitude, location.longitude, address
        else:
            print("La dirección proporcionada no está en Madrid. Intente nuevamente.")

def get_nearest_station(user_location, station_data):
    return min(station_data, key=lambda station: geodesic(user_location[:2], (station['latitude'], station['longitude'])).kilometers)

def visualize_map(user_location, nearest_station):
    my_map = folium.Map(location=(user_location[0], user_location[1]), zoom_start=15)

    folium.Marker(
        location=(user_location[0], user_location[1]),
        popup=folium.Popup("Ubicación del Usuario", parse_html=True),
        icon=folium.Icon(color='blue')
    ).add_to(my_map)

    folium.Marker(
        location=(nearest_station['latitude'], nearest_station['longitude']),
        popup=folium.Popup(f"Estación {nearest_station['station_id']}<br>Dirección: {nearest_station['address']}", parse_html=True),
        icon=folium.Icon(color='red')
    ).add_to(my_map)

    display(my_map)

# Cargamos el DataFrame con los datos reales
data_series = pd.read_csv("../data/interim/bicimad_time_series.csv", sep=',')

# Asegúrate de ajustar el nombre de las columnas según tus datos reales
station_data = [create_station_data(row) for _, row in data_series.iterrows() if create_station_data(row)]

# Obtenemos la ubicación del usuario
user_location = get_user_location()

# Encontramos la estación más cercana
nearest_station = get_nearest_station(user_location, station_data)

# Visualizamos la estación más cercana y la ubicación del usuario en un mapa con Folium
visualize_map(user_location, nearest_station)

La dirección proporcionada no está en Madrid. Intente nuevamente.
La dirección proporcionada no está en Madrid. Intente nuevamente.


# Función para calcular la huella ecológica y el gasto de calorías por trayecto (Según user input)

In [7]:
def calculate_carbon_emission_and_calories(distance, transportation_mode, fuel_type='gasoline'):
    # Factores de emisión y consumo predefinidos para diferentes modos de transporte y tipos de combustible
    emission_factors = {'bicycle': 0.0, 'bus': 0.1, 'car': {'gasoline': 2.35, 'diesel': 2.64}}
    consumption_factors = {'gasoline': 5.4, 'diesel': 4.8}  # L/100km
    
    emission_factor_car = emission_factors.get('car', {}).get(fuel_type, 0.0)
    emission_factor_bicycle = emission_factors.get('bicycle', 0.0)
    
    # Calcular emisiones de CO2 para automóvil y bicicleta
    total_emission_car = emission_factor_car / 100 * consumption_factors.get(fuel_type, 0.0) * distance
    total_emission_bicycle = emission_factor_bicycle * distance
    
    # Calcular el ahorro de emisiones al usar la bicicleta en lugar del automóvil
    emission_savings = total_emission_car - total_emission_bicycle
    
    # Calcular calorías gastadas (supongamos una velocidad promedio de 20 km/h)
    speed_kmh = 20
    if transportation_mode == 'bicycle':
        if speed_kmh <= 15:
            total_calories = 300 * distance
        elif speed_kmh <= 18:
            total_calories = 420 * distance
        elif speed_kmh <= 22:
            total_calories = 600 * distance
        elif speed_kmh <= 28:
            total_calories = 850 * distance
        else:
            total_calories = 1000 * distance
    else:
        total_calories = 0.0
    
    # Convertir calorías a kilocalorías (1 Cal = 1 kcal)
    total_calories_kcal = total_calories / 1000
    
    return emission_savings, total_calories_kcal

# Ejemplo de uso:
distance_traveled = 10  # Supongamos una distancia de 10 kilómetros
transportation_mode_used = 'bicycle'  # Cambia a 'bus' o 'car' según sea necesario
fuel_type_used = 'gasoline'  # Cambia a 'diesel' si es necesario

emission_savings, calories_kcal = calculate_carbon_emission_and_calories(distance_traveled, transportation_mode_used, fuel_type_used)
print(f"Ahorro de emisiones al usar la bicicleta en lugar del coche: {emission_savings:.2f} kg")
print(f"Calorías gastadas: {calories_kcal:.2f} kcal")



Ahorro de emisiones al usar la bicicleta en lugar del coche: 1.27 kg
Calorías gastadas: 6.00 kcal


In [8]:
import folium
import networkx as nx
import pandas as pd
import webbrowser
from geopy.geocoders import Nominatim

# Bloque 1: Crear el Grafo
G = nx.DiGraph()

# Agregar nodos al grafo (estaciones)
for index, row in data_series.iterrows():
    G.add_node(row['station_unlock'], latitude=row['latitude_unlock'], longitude=row['longitude_unlock'])
    G.add_node(row['station_lock'], latitude=row['latitude_lock'], longitude=row['longitude_lock'])

# Agregar arcos ponderados al grafo (distancias entre estaciones)
for index, row in data_series.iterrows():
    G.add_edge(row['station_unlock'], row['station_lock'], distance=row['distance'], time=row['trip_minutes'])

# Bloque 2: Funciones Auxiliares
def get_time_adjustment(current_time, current_day_of_week):
    # Implementa la lógica para ajustar el tiempo de viaje según la hora actual y el día de la semana
    # Puedes personalizar esta función según tus necesidades
    return 1.0

def is_bike_available(station_id):
    # Implementa la lógica para verificar la disponibilidad de bicicletas en una estación
    # Puedes personalizar esta función según tus necesidades
    return True

# Bloque 3: Función para Calcular y Visualizar la Ruta Óptima
def optimize_route(source_station, target_station, current_time, current_day_of_week, max_distance, max_time):
    def edge_weight(u, v, data):
        distance = G[u][v]['distance']
        time = G[u][v]['time']
        time *= get_time_adjustment(current_time, current_day_of_week)
        if not is_bike_available(v):
            return float('inf')
        return distance + time

    path = nx.shortest_path(G, source=source_station, target=target_station, weight=edge_weight)

    total_distance = sum(G[path[i]][path[i+1]]['distance'] for i in range(len(path)-1))
    total_time = sum(G[path[i]][path[i+1]]['time'] for i in range(len(path)-1))

    if total_distance > max_distance or total_time > max_time:
        return None

    return path

# Bloque 4: Visualizar la Ruta en el Mapa
def visualize_route_on_map(graph, path):
    geolocator = Nominatim(user_agent="my_geocoder")
    my_map = folium.Map(location=(graph.nodes[path[0]]['latitude'], graph.nodes[path[0]]['longitude']), zoom_start=15)

    for station in path:
        lat, lon = graph.nodes[station]['latitude'], graph.nodes[station]['longitude']

        # Obtener el nombre de la calle utilizando Nominatim
        location = geolocator.reverse((lat, lon), language='es')
        street_name = location.address if location else f"Lat: {lat}, Lon: {lon}"

        folium.Marker(location=[lat, lon], popup=f"{street_name} (Station {station})").add_to(my_map)

    for i in range(len(path) - 1):
        start_latlon = (graph.nodes[path[i]]['latitude'], graph.nodes[path[i]]['longitude'])
        end_latlon = (graph.nodes[path[i + 1]]['latitude'], graph.nodes[path[i + 1]]['longitude'])
        folium.PolyLine([start_latlon, end_latlon], color="blue", weight=2.5, opacity=1).add_to(my_map)

    # Visualizar el mapa directamente sin guardar el archivo HTML
    my_map.save('ruta_optima.html')
    webbrowser.open('ruta_optima.html')

# Bloque 5: Ejemplo de Uso
source_station = 1
target_station = 2
max_distance = 100
max_time = 60

# Asignar valores para current_time y current_day_of_week
current_time = 12  # Hora actual (ajustar según sea necesario)
current_day_of_week = 'Mon'  # Día de la semana actual (ajustar según sea necesario)

result = optimize_route(source_station, target_station, current_time, current_day_of_week, max_distance, max_time)

if result:
    path = result
    print("Ruta óptima:")
    for i, station in enumerate(path):
        print(f"{i + 1}. Estación {station}")

    visualize_route_on_map(G, path)
else:
    print("No se encontró una ruta que cumpla con las restricciones.")



Ruta óptima:
1. Estación 1
2. Estación 175
3. Estación 2


Modelo predictivo Random Forest

Librerias

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

Cargar datos

In [36]:
data_series = pd.read_csv("../data/interim/bicimad_time_series.csv", sep=',')